In [1]:
pip install spacy-udpipe

     |████████████████████████████████| 5.9 MB 9.8 MB/s 
     |████████████████████████████████| 304 kB 69.2 MB/s 
     |████████████████████████████████| 10.1 MB 37.5 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 623 kB 55.4 MB/s 
     |████████████████████████████████| 456 kB 68.0 MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626680 sha256=11b2bcc2c177902c80e429add3bba7031229fd9ef293870a2006169197eb34aa
  Stored in directory: /root/.cache/pip/wheels/b8/b5/8e/3da091629a21ce2d10bf90759d0cb034ba10a5cf7a01e83d64
Successfully built ufal.udpipe
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc

In [2]:
import spacy_udpipe

spacy_udpipe.download("ru") 

Downloaded pre-trained UDPipe model for 'ru' language


In [3]:
from lxml import etree

In [4]:
from typing import List, Tuple

In [5]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = str(column.text).strip()
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels

In [7]:
texts, labels = load_sentirueval_2016('tkk_train_2016.xml')

In [8]:
print('Number of texts is {0}, number of labels is {1}.'.format(len(texts), len(labels)))

Number of texts is 8643, number of labels is 8643.


In [10]:
nlp = spacy_udpipe.load("ru")

In [11]:
lemmatized_texts = []
for text in texts:
  doc = nlp(text)
  lemmatized_text = []
  for token in doc:
    lemmatized_text.append(token.lemma_)
  lemmatized_texts.append(lemmatized_text)

In [12]:
import random

In [13]:
for idx in random.choices(list(range(len(texts))), k=20):
    print('{0} => {1}'.format(labels[idx], texts[idx]))

negative => RT @tivodivageny: Чёртов кривой модем от билайна, опять он весь вечер будет меня дисконектить, если быстро качаю. Ненавижу.
neutral => "Билайн" начинает продажи iPhone 5s и iPhone 5c в России с 25 октября
neutral => @me42005 Здравствуйте!Запрет действует на sms c коротких номеров и устанавливается только по желанию абонента. #билайн
neutral => @Kalars07 у меня нет билайновских симок, увы(
neutral => @gorejoda Ограничений не зафиксировано. Для детальной проверки, пож-та, пришлите Ваш номер и ссылку на твит на pomogite@beeline.ru. #билайн
neutral => @natashka_2696 проведем детальную проверку и свяжемся с Вами. ^СН #МТС
neutral => В салоне МТС какому то гопнику отказали в кредите на iphone... Его реакция положила со смеху всех: "Отказали? Чё в натуре шоли???"
neutral => МТС готов дать айфонам LTE http://t.co/Jz19hrUXcl
positive => "Вымпелком" передал данные за границу // Связь.Несмотря на падение выездного турпотока на 25%, интернет-трафик абонентов "Вымпелкома" тольк
positive

In [14]:
positive_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'positive']
negative_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'negative']

In [15]:
for cur in positive_tweets[:20]: print(cur)

ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехом МТС - Супер 0
«МегаФон» поможет контролировать расходы на Интернет за границей
УрБК, Екатеринбург, 22.07.2014. ОАО «МегаФон» стало предоставлять услуги сверхскоростной мобильной связи 4G+ еще в 14 населенных пунктах Св
«МегаФон» представил новую возможность сэкономить в роуминге
Завтра может у меня будет и Домашнее Цифровое ТВ МТС #RT
@Beeline_RUS ребят, касательно вашего чудесного Мой Билайн.app для iPhone. А можно его сделать ещё чуточку чудеснее?) http://t.co/MgNY0TdPpt
@Tselyanov @diden05 @redcom_internet Им и пользуюсь сейчас! Спасибо @MegaFonDVpr !
завтра хочу сделать фотку с Сашей из мтс ,реально !! уу,Женя меня убьет))
Возможность быть "онлайн" - специфика работы требует - очень радует (теперь ещё вафай-переносной-модем куплю мегафоновский и вообще в
RT @_Beeline_kz: Сегодня День Системного Администратора! =) С праздником, друзья! http://t.co/KiDNwq11ht
«МегаФон» в Вологодской области: полгода на скоростях 4G+
«МТС Урал» отмечает 

In [16]:
for cur in negative_tweets[:20]: print(cur)

RT @fuckkiev: “@EvaKobb: МТС Россия прислала жителям Херсонщины сообщения, в которых обозвала украинцев фашистами? http://t.co/RbSesXlOUZ” …
@parfenov1960 потому что МТС достало, а пчел ненавижу с детства, как и их мёд!
RT @f_u_c_k_y_o_u_: Билайн интернет стал полным говном
@parfenov1960 Фуфло эт ваш Мегафон.просит много,дает мало.
Вот если и хаять сеть, то МТС! Просто адский оператор какой-то!
RT @51evgen: @EvaKobb давайте откажемся от МТС,у меня 4 номера Я ГОТОВ!
@imerkouri враньё! #Мегафон ГЭ! Проверено!
Что объединяет #МТС и #МГТС ? Нет,не то, что они в одной группе компаний,а одинаково низкое качество услуг и поддержки
#Траффик на Билайне уходит неумолимо :(
#Билайн #Москва http://t.co/KGRzUG10xO
RT @owihycady: Новая реклама от пчелайна: - "Просто пополняйте счет, и говорите бесплатно!". Я один вижу здесь какой-то подвох?
@ru_mts вот утомили вы своей хитрожопостью, но пока альтернативы в местности нет, буду Вас заебывать.
Завтра придется идти в Мегафон, а то я что-то не разберу
Ин

In [17]:
from nltk import word_tokenize

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
vectorizer = CountVectorizer(lowercase=True, tokenizer=word_tokenize)

In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
vectorizer.fit(texts)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x7f75bed169e0>,
                vocabulary=None)

In [101]:
print(vectorizer.get_feature_names()[50:80])

['+79871547330', '+wifi', '+доплата', ',', '-', '-*', '-+,4', '--', '-.-', '-/', '-1100', '-117', '-1500.', '-198', '-2', '-2.15', '-220', '-35', '-36', '-400', '-60', '-60000', '-77', '-d', '-http', '-p', '-а', '-ага', '-бесплатно', '-больше']


In [86]:
from nltk import TreebankWordTokenizer

In [87]:
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2), tokenizer=TreebankWordTokenizer().tokenize)

In [88]:
X_2 = bigram_vectorizer.fit_transform(texts).toarray()

In [89]:
analyze = bigram_vectorizer.build_analyzer()

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer

In [90]:
transformer_2 = TfidfTransformer().fit(X_2)

In [91]:
X_2_transformed = transformer_2.transform(X_2)

In [27]:
print(texts[0])

@mkomov Максим, Вашем письмо мы получили. Наши сотрудники свяжутся с Вами завтра и направят запрос инженерам для проверки. #билайн


In [29]:
print(vectorizer.get_feature_names()[3739])

@


In [92]:
print(X_2_transformed[0])

  (0, 60196)	0.2205518134918478
  (0, 57518)	0.2205518134918478
  (0, 56557)	0.16474523320124843
  (0, 53502)	0.20798618648041095
  (0, 51121)	0.23052571275922895
  (0, 49185)	0.23052571275922895
  (0, 42911)	0.2205518134918478
  (0, 42457)	0.23052571275922895
  (0, 41052)	0.23052571275922895
  (0, 37633)	0.2205518134918478
  (0, 33412)	0.23052571275922895
  (0, 32244)	0.23052571275922895
  (0, 31098)	0.23052571275922895
  (0, 30725)	0.2205518134918478
  (0, 28297)	0.1997094195596056
  (0, 23812)	0.23052571275922895
  (0, 23581)	0.23052571275922895
  (0, 16300)	0.23052571275922895
  (0, 11977)	0.23052571275922895
  (0, 2181)	0.23052571275922895
  (0, 629)	0.09378773384322041


In [74]:
print(bigram_vectorizer.get_feature_names()[15081])

вами завтра


In [93]:
tokens_with_IDF_2 = list(zip(bigram_vectorizer.get_feature_names(), transformer_2.idf_))

In [94]:
for feature, idf in tokens_with_IDF_2[0:20]: print('{0:.3f} => {1}'.format(idf, feature))

5.123 => ! !
5.480 => ! #
8.966 => ! &
6.929 => ! ''
7.580 => ! (
6.769 => ! )
9.371 => ! *110*1111
8.455 => ! +
9.371 => ! +7
9.371 => ! +79111812983
8.966 => ! ,
8.273 => ! -
9.371 => ! -всем
8.678 => ! .
8.966 => ! ..
8.678 => ! ...
7.762 => ! /
9.371 => ! 1
9.371 => ! 14
9.371 => ! 1911


In [32]:
from sklearn.feature_selection import SelectPercentile, chi2

In [33]:
selector = SelectPercentile(chi2, percentile=20)

In [95]:
selector.fit(X_2_transformed, labels)

SelectPercentile(percentile=20, score_func=<function chi2 at 0x7f75bde06170>)

In [96]:
selected_tokens_with_IDF_2 = [tokens_with_IDF_2[idx] for idx in selector.get_support(indices=True)]

In [97]:
for feature, idf in selected_tokens_with_IDF_2[0:20]: print('{0:.3f} => {1}'.format(idf, feature))

5.123 => ! !
6.929 => ! ''
7.580 => ! (
6.769 => ! )
8.455 => ! +
8.966 => ! ,
9.371 => ! -всем
9.371 => ! 14
9.371 => ! 40
9.371 => ! 4г
6.846 => ! :
9.371 => ! =
8.966 => ! ``
5.845 => ! http
9.371 => ! lte
9.371 => ! «
9.371 => ! ааааеее
9.371 => ! бесплатно
8.678 => ! больше
9.371 => ! вечно


In [98]:
selected_and_sorted_tokens_with_IDF_2 = sorted(selected_tokens_with_IDF_2, key=lambda it: (-it[1], it[0]))

In [99]:
for feature, idf in selected_and_sorted_tokens_with_IDF_2[0:40]: print('{0:.3f} => {1}'.format(idf, feature))

9.371 => ! -всем
9.371 => ! 14
9.371 => ! 40
9.371 => ! 4г
9.371 => ! =
9.371 => ! lte
9.371 => ! «
9.371 => ! ааааеее
9.371 => ! бесплатно
9.371 => ! вечно
9.371 => ! выручили
9.371 => ! голосуем
9.371 => ! даже
9.371 => ! делитесь
9.371 => ! добро
9.371 => ! доступ
9.371 => ! думаю
9.371 => ! епт
9.371 => ! ждем
9.371 => ! заправляйтесь
9.371 => ! звонили
9.371 => ! ибо
9.371 => ! кирдык
9.371 => ! мобильный
9.371 => ! молодцы
9.371 => ! мтс-олдскул
9.371 => ! назвали
9.371 => ! нам
9.371 => ! ненавижу
9.371 => ! нет
9.371 => ! оно
9.371 => ! отличная
9.371 => ! отлично
9.371 => ! отличный
9.371 => ! паника
9.371 => ! перезагрузите
9.371 => ! плюс
9.371 => ! поднимает
9.371 => ! получите
9.371 => ! прислал


In [100]:
for feature, idf in selected_and_sorted_tokens_with_IDF_2[-40:-1]: print('{0:.3f} => {1}'.format(idf, feature))

5.543 => на pomogite
5.532 => мегафон ''
5.521 => , как
5.521 => » и
5.480 => pomogite @
5.470 => ? #
5.440 => « мтс-украина
5.420 => мтс-украина »
5.382 => ростелеком »
5.373 => « ростелеком
5.320 => здравствуйте !
5.302 => мтс украина
5.277 => билайн http
5.269 => : )
5.205 => , но
5.205 => @ megafonru
5.167 => `` билайн
5.159 => билайн ''
5.123 => ! !
5.109 => ) )
5.081 => & amp
5.081 => amp ;
5.067 => мтс http
5.021 => : http
4.971 => мтс ,
4.797 => в крыму
4.792 => , пожалуйста
4.747 => пожалуйста ,
4.713 => билайн »
4.689 => « билайн
4.649 => « мегафон
4.649 => мегафон »
4.631 => , а
4.622 => у меня
4.523 => @ ru_mts
4.248 => , что
3.866 => # мтс
3.813 => # билайн
2.985 => rt @


Можно ввести бинарный признак наличия таких биграм, как
- ":)" - положительная эмоциональная окраска;
- "!!" - не нейтральная эмоциональная окраска;
- "))" - положительная эмоциональная окраска;
- "здравствуйте!" - не негативная эмоциональная окраска, потому что мало кто начинает гневные твиты с пожелания здоровья...